In [1]:
import pandas as pd
import mysql.connector
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
from datetime import datetime
import os

# Configuración de conexión a la base de datos
DB_CONFIG = {
    "host": "roundhouse.proxy.rlwy.net",
    "port": 38517,
    "user": "root",
    "password": "wZGotyxIDqpAtQjPxNuxxezSbbroztiw",
    "database": "railway"
}

# Crear carpeta para guardar las gráficas
os.makedirs("graficas_comparativas", exist_ok=True)

# Conexión y carga de datos
conn = mysql.connector.connect(**DB_CONFIG)
query = """
SELECT TipoMantenimiento, Fecha, Costo, Origen
FROM Predicciones_Tipo_Mantenimiento
"""
df = pd.read_sql(query, conn)
conn.close()

# Preprocesamiento
df['Fecha'] = pd.to_datetime(df['Fecha'])
df = df.sort_values(by='Fecha')

# Filtrar últimos 12 meses por origen
df_historico = df[df['Origen'] == 'Histórico'].groupby('TipoMantenimiento').apply(lambda x: x.sort_values('Fecha').tail(12)).reset_index(drop=True)
df_prediccion = df[df['Origen'] == 'Predicción'].groupby('TipoMantenimiento').apply(lambda x: x.sort_values('Fecha').tail(12)).reset_index(drop=True)
df_filtrado = pd.concat([df_historico, df_prediccion], ignore_index=True)

# Comparación mes a mes
df_pivot = df_filtrado.pivot_table(index=['TipoMantenimiento', 'Fecha'], columns='Origen', values='Costo').reset_index()
df_pivot['Diferencia'] = df_pivot['Predicción'] - df_pivot['Histórico']
df_pivot['% Cambio'] = ((df_pivot['Predicción'] - df_pivot['Histórico']) / df_pivot['Histórico']) * 100

# Totales por origen
totales = df_filtrado.groupby(['TipoMantenimiento', 'Origen'])['Costo'].sum().unstack()
totales['Diferencia'] = totales['Predicción'] - totales['Histórico']
totales['% Cambio'] = ((totales['Predicción'] - totales['Histórico']) / totales['Histórico']) * 100

# Gráfica comparativa en barras por mes
for tipo in df_pivot['TipoMantenimiento'].unique():
    df_tipo = df_pivot[df_pivot['TipoMantenimiento'] == tipo]
    x = df_tipo['Fecha'].dt.strftime('%b %Y')
    x_idx = range(len(x))
    width = 0.35

    fig, ax = plt.subplots(figsize=(14, 6))
    bars1 = ax.bar([i - width/2 for i in x_idx], df_tipo['Histórico'], width=width, label='Histórico')
    bars2 = ax.bar([i + width/2 for i in x_idx], df_tipo['Predicción'], width=width, label='Predicción')

    for i, bar in enumerate(bars1):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2, height / 2, f"${height:,.0f}", ha='center', va='center', color='white', fontsize=9, rotation=90)
    for i, bar in enumerate(bars2):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2, height / 2, f"${height:,.0f}", ha='center', va='center', color='white', fontsize=9, rotation=90)

    ax.set_title(f'Comparativo Histórico vs Predicción - Tipo: {tipo}')
    ax.set_xlabel('Fecha')
    ax.set_ylabel('Costo ($)')
    ax.set_xticks(x_idx)
    ax.set_xticklabels(x, rotation=45)
    ax.legend()
    ax.grid(True, axis='y')
    ax.yaxis.set_major_formatter(ticker.StrMethodFormatter('${x:,.0f}'))
    plt.tight_layout()
    plt.savefig(f"graficas_comparativas/comparativo_{tipo.replace(' ', '_')}.png", bbox_inches='tight')
    plt.show()

# Gráfica de totales generales
for tipo in totales.index:
    total_hist = totales.loc[tipo, 'Histórico']
    total_pred = totales.loc[tipo, 'Predicción']

    fig, ax = plt.subplots(figsize=(6, 5))
    bars = ax.bar(['Histórico', 'Predicción'], [total_hist, total_pred], color=['#1f77b4', '#ff7f0e'])
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2, height / 2, f"${height:,.0f}", ha='center', va='center', color='white', fontsize=10, rotation=90)

    ax.set_title(f'Total Histórico vs Predicción - Tipo: {tipo}')
    ax.set_ylabel('Costo Total ($)')
    ax.yaxis.set_major_formatter(ticker.StrMethodFormatter('${x:,.0f}'))
    ax.grid(True, axis='y')
    plt.tight_layout()
    plt.savefig(f"graficas_comparativas/total_{tipo.replace(' ', '_')}.png", bbox_inches='tight')
    plt.show()

# Generar solo dos gráficas tipo torta con resumen: correctivo y preventivo
for tipo in ['CORRECTIVO', 'PREVENTIVO']:
    if tipo in totales.index:
        total_hist = totales.loc[tipo, 'Histórico']
        total_pred = totales.loc[tipo, 'Predicción']
        diferencia = total_pred - total_hist
        porcentaje_cambio = (diferencia / total_hist) * 100
        cambio = 'Crecimiento' if diferencia >= 0 else 'Reducción'

        valores = [total_hist, abs(diferencia)]
        etiquetas = ['Histórico', cambio]
        colores = ['#1f77b4', '#2ca02c' if diferencia >= 0 else '#d62728']

        fig, ax = plt.subplots(figsize=(6, 6))
        wedges, texts, autotexts = ax.pie(valores, labels=etiquetas, autopct='%1.1f%%', startangle=90, colors=colores)
        ax.set_title(f'Participación Histórica vs {cambio} - Tipo: {tipo}')

        porcentaje_grafica = autotexts[1].get_text()

        resumen = (
            f"Histórico: ${total_hist:,.0f}\n"
            f"Predicción: ${total_pred:,.0f}\n"
            f"{cambio}: ${abs(diferencia):,.0f} ({porcentaje_cambio:+.1f}%)\n"
        )
        resumen += f"El porcentaje de {cambio.lower()} fue de {porcentaje_grafica}"

        # Ajuste para mostrar bien el resumen
        plt.subplots_adjust(bottom=0.25)
        plt.figtext(0.5, -0.05, resumen, ha="center", fontsize=10)

        plt.savefig(f"graficas_comparativas/torta_{cambio.lower()}_{tipo}.png", bbox_inches='tight')
        plt.show()

# Mostrar resultados
print("\nComparativo Mensual:")
display(df_pivot)

print("\nTotales por Origen:")
display(totales)


C:\Users\adrif\AppData\Local\Temp\ipykernel_19088\3274894273.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
C:\Users\adrif\AppData\Local\Temp\ipykernel_19088\3274894273.py:35: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_historico = df[df['Origen'] == 'Histórico'].groupby('TipoMantenimiento').apply(lambda x: x.sort_values('Fecha').tail(12)).reset_index(drop=True)
C:\Users\adrif\AppData\Local\Temp\ipykernel_19088\3274894273.py:36: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This be

KeyError: 'Histórico'